In [1]:
import os
import glob
import pandas as pd
import re

In [2]:
import glob

# Ruta a la carpeta que contiene los CSV's
RawData_rstate = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM'

# Obtener todos los archivos .csv en el directorio RawData
csv_files = glob.glob(os.path.join(RawData_rstate, '*.csv'))

# Lista para almacenar rutas completas de archivo
file_paths = []

# Iterar sobre los archivos y obtener las rutas completas
for file in csv_files:
    file_paths.append(file)

# Imprimir la lista de rutas completas de archivo
print(file_paths)

['/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_sevilla_20240709.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_cadiz_20240626.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_malaga_20240626.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_granada_20240705.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_madrid_20240708.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TF

In [3]:
# Diccionario para almacenar los dataframes con sus nombres de archivo
dataframes = {}

# Iterar sobre cada archivo CSV y crear un dataframe
for file in csv_files:
    # Extraer el nombre del archivo sin la ruta y la extensión
    file_name = file.split('/')[-1].replace('.csv', '')
    try:
        # Leer el archivo CSV en un dataframe de pandas
        df = pd.read_csv(file, delimiter=';', on_bad_lines='skip')
        
        # Determinar valor para columna 'fuente_datos'
        if 'trovit' in file_name:
            df['fuente_datos'] = 'trovit'
        else:
            df['fuente_datos'] = 'pisos.com'
        
        # Determinar valor para columna 'alquiler_venta'
        if 'alquiler' in file_name:
            df['alquiler_venta'] = 'alquiler'
        else:
            df['alquiler_venta'] = 'venta'
        
        # Determinar valor para columna 'fecha_extract'
        # Extraer la parte de la fecha del nombre del archivo
        fecha_extract_str = file_name.split('_')[-1]  # Obtener '20240630', '20240630', etc.
        # Convertir a formato de fecha
        fecha_extract = pd.to_datetime(fecha_extract_str, format='%Y%m%d')
        df['fecha_extract'] = fecha_extract
        
        # Almacenar el dataframe en el diccionario
        dataframes[file_name] = df
        
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")

In [4]:
# Convertir el diccionario de dataframes en una lista de dataframes
dfs = list(dataframes.values())

# Concatenar todos los dataframes en uno solo
consolidated_df = pd.concat(dfs, ignore_index=True)

# Mostrar el dataframe consolidado
print(consolidated_df.tail())

             precio                sub_descr  \
236652  1.200 €/mes                      NaN   
236653  1.500 €/mes                      NaN   
236654    800 €/mes                      NaN   
236655  2.500 €/mes  Piso en calle San Pablo   
236656  1.125 €/mes                      NaN   

                                                     href  \
236652  /alquilar/piso-zona_esperanza_de_triana41010-4...   
236653   /alquilar/atico-alfalfa41002-39214413380_525060/   
236654  /alquilar/piso-los_remedios_barrio41011-450523...   
236655      /alquilar/piso-museo41001-45096394992_100500/   
236656  /alquilar/piso-barrio_leon_el_tardon_blas_infa...   

                                                ubicacion habitaciones  \
236652  Zona Esperanza de Triana (Distrito Triana. Sev...      4 habs.   
236653  Alfalfa (Distrito Casco Antiguo. Sevilla Capital)      3 habs.   
236654  Los Remedios (Distrito Los Remedios. Sevilla C...       1 hab.   
236655    Museo (Distrito Casco Antiguo. Sevilla

def extraer_numeros(columna):
    numeros_extraidos = columna.str.extract(r'(\d+\.?\d*)').astype(float)
    return numeros_extraidos


def extraer_numeros_decimales(columna):
    numeros_extraidos = columna.str.replace(',', '.')
    numeros_extraidos = numeros_extraidos.astype('float')
    return numeros_extraidos

def limpia_datos(dat):
    # Elimina duplicados
    dat.drop_duplicates(inplace = True)
    
    # Latitud y longitud a numérico
    dat['latitud_numerico'] = extraer_numeros_decimales(dat['latitude'])
    dat['longitud_numerico'] = extraer_numeros_decimales(dat['longitude'])
    
    # Extraemos los números de las columnas 'habitaciones', 'banios' y 'mt2'
    dat['habitaciones_numerico'] = extraer_numeros(dat['habitaciones'])
    dat['banios_numerico'] = extraer_numeros(dat['banios'])
    dat['mt2_numerico'] = extraer_numeros(dat['mt2'])
    
    dat['precio_numerico'] = dat['precio'].str.replace('.', '').str.extract(r'(\d+)').astype(float)
    # Chequear si todos los precios están en euros
    
    
    dat['frase_paréntesis'] = dat['ubicacion'].str.extract(r'\((.*?)\)')
    
    # Separamos la frase extraída utilizando el punto como separador
    dat[['Distrito', 'Otra_ubicacion']] = dat['frase_paréntesis'].str.split('.', n=1, expand=True)
    
    # Eliminamos los espacios en blanco alrededor de las cadenas resultantes
    dat['Distrito'] = dat['Distrito'].str.strip()
    dat['Otra_ubicacion'] = dat['Otra_ubicacion'].str.strip()
    
    # Eliminamos la columna temporal 'frase_paréntesis'
    dat.drop(columns=['frase_paréntesis'], inplace=True)
    
    # Generamos una nueva columna 'precio_por_mt2' que representa el precio por metro cuadrado
    dat['precio_por_mt2'] = dat['precio_numerico'] / dat['mt2_numerico']
    
    
    ## Guarda datos limpios y elimina columnas repetidas
    cols_to_drop = ['precio', 'habitaciones', 'banios',
           'mt2', 'planta', 'latitude', 'longitude']
    dat.drop(columns=cols_to_drop, inplace = True)
    dat.columns = dat.columns.str.replace('_numerico','')
    return dat


columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ]
dat = limpia_datos(consolidated_df[column])

columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ] # Asegúrate de ajustar según tus columnas reales
for column in columns_to_clean_consolidated:
    consolidated_df[column] = consolidated_df[column].apply(limpia_datos)

In [5]:
def limpiar_numeros(numero_en_sucio):
    # Manejar valores NaN y convertir a cadena si es necesario
    if pd.isna(numero_en_sucio):
        return None
    # Convertir a cadena
    numero_en_sucio = str(numero_en_sucio)
    # Eliminar cualquier texto que no sea un número, punto, coma o espacio
    numero_en_sucio = re.sub(r'[^\d.,€\s]', '', numero_en_sucio)
    # Eliminar el símbolo de euro y espacios en blanco
    numero_en_sucio = re.sub(r'[€\s]', '', numero_en_sucio)
    # Reemplazar los puntos (separadores de miles) por nada
    numero_en_sucio = numero_en_sucio.replace('.', '')
    # Reemplazar las comas (separadores decimales) por nada
    numero_en_sucio = numero_en_sucio.replace(',', '')
    # Convertir a entero, manejar excepciones si la conversión falla
    try:
        return int(numero_en_sucio)
    except ValueError:
        return None

# Aplicar la misma transformación al dataframe consolidado
columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace', 'planta' ] # Asegúrate de ajustar según tus columnas reales
for column in columns_to_clean_consolidated:
    consolidated_df[column] = consolidated_df[column].apply(limpiar_numeros)

    
# Convertir las columnas 'latitude' y 'longitude' a tipo float
consolidated_df['latitude'] = pd.to_numeric(consolidated_df['latitude'], errors='coerce')
consolidated_df['longitude'] = pd.to_numeric(consolidated_df['longitude'], errors='coerce')

In [6]:
consolidated_df.describe()

,precio,habitaciones,banios,mt2,latitude,longitude,planta,fecha_extract,publicado_hace
count,2.330120e+05,229445.000000,212261.000000,214056.000000,160232.000000,160232.000000,68810.000000,236657,72753.000000
mean,2.879758e+05,3.054484,2.010676,228.576064,39.357391,-2.676935,2.917149,2024-07-01 03:35:17.449304320,48.022666
min,4.500000e+01,1.000000,1.000000,1.000000,0.000000,-16.749433,1.000000,2024-06-18 00:00:00,1.000000
25%,2.400000e+03,2.000000,1.000000,79.000000,37.124899,-4.282520,1.000000,2024-06-25 00:00:00,10.000000
50%,1.590000e+05,3.000000,2.000000,108.000000,39.493400,-3.621989,2.000000,2024-06-30 00:00:00,21.000000
75%,3.320000e+05,4.000000,2.000000,172.000000,41.381469,-0.405303,4.000000,2024-07-08 00:00:00,30.000000
max,2.450000e+07,140.000000,332.000000,1000000.000000,43.508612,2.762230,20.000000,2024-07-14 00:00:00,2350.000000
std,5.804899e+05,1.623141,1.535383,7069.782461,2.366645,2.666822,2.176673,NaN,202.098720


In [7]:
consolidated_df

,precio,sub_descr,href,ubicacion,habitaciones,banios,mt2,otros,latitude,longitude,planta,CCAA,fuente_datos,alquiler_venta,fecha_extract,publicado_hace,plataforma,raw_json
0,1200.0,NaN,/alquilar/piso-zona_esperanza_de_triana41010-4...,Zona Esperanza de Triana (Distrito Triana. Sev...,3.0,1.0,69.0,¡Alquiler para estudiantes! Contrato mínimo de...,37.379340,-6.003559,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
1,900.0,Piso en calle Ares,/alquilar/piso-bellavista_la_palmera_bellavist...,Bellavista (Distrito Bellavista-La Palmera. Se...,3.0,2.0,105.0,Magnífico piso en urbanización exclusiva: jard...,37.332449,-5.965829,5.0,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
2,1000.0,NaN,/alquilar/chalet-oromana-45028203361_517213/,Oromana (Alcalá de Guadaíra),4.0,2.0,146.0,Fantástico chalet en alquiler de larga tempora...,37.320301,-5.856279,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
3,990.0,NaN,/alquilar/piso-zona_ronda_de_triana_cartuja410...,Zona Ronda de Triana-Cartuja (Distrito Triana....,2.0,1.0,80.0,¡Descubre tu nuevo hogar en el corazón de tria...,37.389190,-6.008260,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
4,1000.0,NaN,/alquilar/piso-nervion_ciudad_jardin41005-4503...,Ciudad Jardín (Distrito Nervión. Sevilla Capital),3.0,1.0,60.0,Piso en alquiler de 70 m² en ciudad jardin sol...,37.380612,-5.961544,NaN,sevilla,pisos.com,alquiler,2024-07-09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236652,1200.0,NaN,/alquilar/piso-zona_esperanza_de_triana41010-4...,Zona Esperanza de Triana (Distrito Triana. Sev...,4.0,2.0,136.0,Alquiler en triana con garaje\n Característica...,37.379625,-6.004155,NaN,sevilla,pisos.com,alquiler,2024-07-10,NaN,NaN,NaN
236653,1500.0,NaN,/alquilar/atico-alfalfa41002-39214413380_525060/,Alfalfa (Distrito Casco Antiguo. Sevilla Capital),3.0,3.0,120.0,"Altura piso 3º, ático superficie total 120 m²,...",37.393445,-6.000533,NaN,sevilla,pisos.com,alquiler,2024-07-10,NaN,NaN,NaN
236654,800.0,NaN,/alquilar/piso-los_remedios_barrio41011-450523...,Los Remedios (Distrito Los Remedios. Sevilla C...,1.0,1.0,45.0,Estupendo apartamento totalmente amueblado en ...,37.374667,-5.998008,2.0,sevilla,pisos.com,alquiler,2024-07-10,NaN,NaN,NaN
236655,2500.0,Piso en calle San Pablo,/alquilar/piso-museo41001-45096394992_100500/,Museo (Distrito Casco Antiguo. Sevilla Capital),3.0,NaN,200.0,El hogar que buscas en sevilla: un fantástico ...,37.390200,-5.998300,2.0,sevilla,pisos.com,alquiler,2024-07-10,NaN,NaN,NaN


In [8]:
consolidated_df.to_csv('consolidated_data.csv', index=False)